In [1]:
%load_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import gconcorde as cce
from gconcorde import GraphicalConcorde, GraphicalConcordeCV

np.random.seed(3)

# true precision matrix
omega = np.zeros((3,3), order="C")
omega[0, 1] = 2.1
omega[1, 2] = -2.1
omega += omega.T
np.fill_diagonal(omega, 3)

# true covariance matrix
sigma = np.linalg.inv(omega)

# observations
data = np.random.multivariate_normal([0]*3, sigma, 500)

# sample covariance matrix
S = np.cov(data, rowvar=False)

In [2]:
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.01 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 5)

In [3]:
# CONCORDe CV

model = GraphicalConcorde(lam=lams[0], epstol=1e-7, maxitr=500, penalize_diag=True)
model.fit(data)

# predict
print("omega_hat:\n %s\n" % (str(model.precision_.toarray())))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(model.hist_[-3:])))
print("iterations: %d" % (len(model.hist_)))
print("successive norm diff: %e" % (model.hist_[-1,0]))
print("objective: %e" % (model.hist_[-1,1]))

omega_hat:
 [[ 2.03380077  1.36279999  0.        ]
 [ 1.36279999  1.8841713  -1.36310986]
 [ 0.         -1.36310986  2.03805204]]

hist: (inner_iter_count, delta_subg, objective)
 [[1.88941057e-05 9.45847264e-01]
 [1.86311386e-05 9.45847253e-01]
 [1.83719341e-05 9.45847241e-01]]

iterations: 500
successive norm diff: 1.837193e-05
objective: 9.458472e-01


In [4]:
model_cv = GraphicalConcordeCV(lams=lams, epstol=1e-7, maxitr=500, penalize_diag=True, val_score='EBIC', gamma=0.1)
model_cv.fit(data)

In [5]:
# CONCORDe CV

print("omega_hat:\n %s\n" % (str(model_cv.precision_.toarray())))
print("hist: (inner_iter_count, delta_subg, objective)\n %s\n" % (str(model_cv.hist_[-3:])))
print("iterations: %d" % (len(model_cv.hist_)))
print("successive norm diff: %e" % (model_cv.hist_[-1,0]))
print("objective: %e \n" % (model_cv.hist_[-1,1]))
print("cv results:\n %s\n" % pd.DataFrame(model_cv.cv_results_))
print("best lam: %s" % model_cv.best_param_)

omega_hat:
 [[ 2.03380077  1.36279999  0.        ]
 [ 1.36279999  1.8841713  -1.36310986]
 [ 0.         -1.36310986  2.03805204]]

hist: (inner_iter_count, delta_subg, objective)
 [[1.88941057e-05 9.45847264e-01]
 [1.86311386e-05 9.45847253e-01]
 [1.83719341e-05 9.45847241e-01]]

iterations: 500
successive norm diff: 1.837193e-05
objective: 9.458472e-01 

cv results:
          lam         score
0   0.116869   7800.831566
1   0.369572   8565.759128
2   1.168689   8934.402873
3   3.695718  12894.612277
4  11.686885  12894.612277

best lam: 0.11686885357906268
